# notes

* take the data
* normalize

still have to decide whether to separate the files or not

# SetUp

In [1]:
# LOAD LIBRARIES
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(future))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(presto))
suppressPackageStartupMessages(library(cowplot))
suppressPackageStartupMessages(library(tictoc))

suppressPackageStartupMessages(library(enrichR))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(AnnotationDbi))

suppressPackageStartupMessages(library(SingleR))

suppressPackageStartupMessages(library(GPTCelltype))
suppressPackageStartupMessages(library(openai))

In [123]:
total_time <- function(seconds) {
    d <- seconds %/% (86400)
    h <- (seconds %% 86400) %/% 3600
    m <- (seconds %% 3600) %/% 60
    s <- seconds %% 60
    
    cat(sprintf("Total Time: %f Days, %f Hours, %f Minutes and %f Seconds\n", d, h, m, s))
}

# Seurat

## Functions

### Load data

In [80]:
load.data <- function(
    file_name,
    data_path = path_to_data,
    output = F,
    reduced.output = T 
) {
    # Load the data
    data <- Read10X(data.dir = paste(data_path, "expression_", file_name, sep = ""), gene.column = 1)

    return(data)
}

### Preprocessing

In [12]:
preprocessing <- function(
    data,
    output = F
) {
    # Create Seurat object
    sc_data <- CreateSeuratObject(counts = data, min.cells = 3, min.features = 500, names.delim = "-", names.field = 2)

    # Normalize the data
    sc_data <- NormalizeData(sc_data, normalization.method = "LogNormalize", scale.factor = 1e6, verbose = output)

    # Find variable features
    sc_data <- FindVariableFeatures(sc_data, selection.method = "mvp", nfeatures = 2000, verbose = output)

    # Scale the data
    sc_data <- ScaleData(sc_data, verbose = output)

    return(sc_data)
}

### Clusteing

In [46]:
PCA.cluster <- function(
    data, 
    res = 1, 
    n_dim = 40, 
    save = F,
    output = F   
) {
    # PCA
    data <- RunPCA(data, npcs = max(n_dim), verbose = output)
    data <- RunUMAP(data, dims = n_dim, verbose = output)
    #print(ElbowPlot(object = data, ndims = max(50, n_dim)))

    # Cluster the cells
    data <- FindNeighbors(data, dims = n_dim, verbose = output)
    data <- FindClusters(data, resolution = res, verbose = output)
    
    #print(table(Idents(data)))

    # Save the parial
    if (save != F) {
        name_new_dir <- paste("Partial/", save, "/cluster", param, sep="")
        if (!dir.exists(name_new_dir)) {dir.create(name_new_dir)} 
    
        print(paste("Saving PCA for time point", save, "in", name_new_dir))
        save(data, file = paste(name_new_dir, "/PCA_res_", res, "_dim_", n_dim, "_", save, ".Robj", sep=""))
    }
    
    return(data)
}

### UMAP & PCA plot

In [93]:
cluster.plot <- function(
    data,
    file_name = timepoints[time_point],
    pt.size = 1,
    print_plot = F
) { 
    plots <- list(
        UMAP = DimPlot(data, reduction = "umap", label = TRUE, pt.size = pt.size) + 
            ggtitle(paste("UMAP -",file_name)),
        PCA = DimPlot(data, reduction = "pca", label = TRUE, pt.size = pt.size) + 
            ggtitle(paste("PCA -",file_name))
    )

    if (print_plot) {print(plots)}
    
    return(plots)
}

### Markers

In [33]:
cluster.markers <- function(
    data,
    output = F
) {
    # Find all markers for every cluster compared to all remaining cells
    markers <- FindAllMarkers(data,
                              only.pos = TRUE,   # Considera solo i marker espressi positivamente
                              min.pct = 0.25,    # Percentuale minima di espressione nelle cellule del cluster
                              logfc.threshold = 0.25,  # Soglia minima di LogFC
                              verbose = output)
        
    return(markers)
}

### Top Genes

In [70]:
top.genes <- function(
    clmark = cluster_markers,
    num = 50
) {
    top_genes_by_cluster <- cluster_markers %>% group_by(cluster) %>% top_n(n = top_n_genes, wt = avg_log2FC) %>% as.data.frame()

    return(top_genes_by_cluster)
}

## Do

In [72]:
timepoints = c("23days", "1month", "1.5month", "2month", "3month", "4month", "5month", "6month")
housekeeping_genes = c("ACTB", "DLG4")
genes_of_interest = c("SRCIN1", "KIAA1217", "CIT")
path_to_data = "/sharedFolder/Data/"
res_list = c(1, 0.5, 0.2, 0.1)#seq(0.1, 1, by = 0.1)
dim_list = c(1:20)
top_n_genes = 50

In [95]:
file_name <- timepoints[1]

In [73]:
raw_data <- load.data(file_name)

[1] "Loading data for time point: 23days"


In [74]:
sc_data <- preprocessing(raw_data)
head(sc_data)

,orig.ident,nCount_RNA,nFeature_RNA
,<fct>,<dbl>,<int>
1_AAACGAACACGTGAGA-1_1_23d,1_1_23d,3724,536
1_AAACGAAGTCGAAACG-1_1_23d,1_1_23d,9184,2310
1_AAACGAAGTGGCAGAT-1_1_23d,1_1_23d,6357,2068
1_AAACGAAGTGGTTCTA-1_1_23d,1_1_23d,3468,1292
1_AAACGAAGTTGGCCGT-1_1_23d,1_1_23d,1809,832
1_AAACGAATCCTTCACG-1_1_23d,1_1_23d,7587,1985
1_AAACGCTCAAGCGGAT-1_1_23d,1_1_23d,5924,1600
1_AAACGCTCACTTCAAG-1_1_23d,1_1_23d,2264,1110
1_AAACGCTCATCATCCC-1_1_23d,1_1_23d,9277,2441


In [75]:
sc_data_clustered <- PCA.cluster(sc_data, res = res_list[2], n_dim= dim_list)
summary(sc_data_clustered)
head(sc_data_clustered)

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’



Length  Class   Mode 
     1 Seurat     S4 

,orig.ident,nCount_RNA,nFeature_RNA,RNA_snn_res.0.5,seurat_clusters
,<fct>,<dbl>,<int>,<fct>,<fct>
1_AAACGAACACGTGAGA-1_1_23d,1_1_23d,3724,536,3,3
1_AAACGAAGTCGAAACG-1_1_23d,1_1_23d,9184,2310,3,3
1_AAACGAAGTGGCAGAT-1_1_23d,1_1_23d,6357,2068,2,2
1_AAACGAAGTGGTTCTA-1_1_23d,1_1_23d,3468,1292,2,2
1_AAACGAAGTTGGCCGT-1_1_23d,1_1_23d,1809,832,2,2
1_AAACGAATCCTTCACG-1_1_23d,1_1_23d,7587,1985,3,3
1_AAACGCTCAAGCGGAT-1_1_23d,1_1_23d,5924,1600,6,6
1_AAACGCTCACTTCAAG-1_1_23d,1_1_23d,2264,1110,4,4
1_AAACGCTCATCATCCC-1_1_23d,1_1_23d,9277,2441,2,2


In [96]:
cluster_plots <- cluster.plot(
    sc_data_clustered,
    file_name = file_name,
    pt.size = 0.5,
    print_plot = F
)

In [77]:
cluster_markers <- cluster.markers(sc_data_clustered, output = T)
summary(cluster_markers)
head(cluster_markers)

     p_val             avg_log2FC          pct.1            pct.2       
 Min.   :0.0000000   Min.   : 0.2500   Min.   :0.1670   Min.   :0.0000  
 1st Qu.:0.0000000   1st Qu.: 0.3978   1st Qu.:0.3250   1st Qu.:0.1770  
 Median :0.0000000   Median : 0.6083   Median :0.4350   Median :0.2760  
 Mean   :0.0001703   Mean   : 0.9779   Mean   :0.5015   Mean   :0.3412  
 3rd Qu.:0.0000000   3rd Qu.: 1.0635   3rd Qu.:0.6370   3rd Qu.:0.4470  
 Max.   :0.0099990   Max.   :13.8465   Max.   :1.0000   Max.   :0.9990  
                                                                        
   p_val_adj            cluster         gene          
 Min.   :0.0000000   7      :1339   Length:13730      
 1st Qu.:0.0000000   9      :1232   Class :character  
 Median :0.0000000   8      :1209   Mode  :character  
 Mean   :0.1003849   15     :1197                     
 3rd Qu.:0.0000002   16     :1140                     
 Max.   :1.0000000   12     : 965                     
                     (Other):66

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
DCT,0,1.780295,0.383,0.161,0,0,DCT
AMBN,0,1.767947,0.431,0.175,0,0,AMBN
HEY1,0,1.708512,0.464,0.202,0,0,HEY1
LINC01551,0,1.623801,0.767,0.311,0,0,LINC01551
OLFM3,0,1.543925,0.396,0.189,0,0,OLFM3
SFRP1,0,1.514941,0.968,0.510,0,0,SFRP1


In [79]:
top_genes <- top.genes(top_genes_by_cluster, num = 50)
summary(top_genes)
head(top_genes)

     p_val             avg_log2FC          pct.1            pct.2      
 Min.   :0.000e+00   Min.   : 0.9538   Min.   :0.2500   Min.   :0.000  
 1st Qu.:0.000e+00   1st Qu.: 2.0188   1st Qu.:0.3180   1st Qu.:0.028  
 Median :0.000e+00   Median : 3.0030   Median :0.4150   Median :0.072  
 Mean   :5.964e-47   Mean   : 3.6568   Mean   :0.4775   Mean   :0.113  
 3rd Qu.:0.000e+00   3rd Qu.: 4.2219   3rd Qu.:0.5917   3rd Qu.:0.149  
 Max.   :5.064e-44   Max.   :13.8465   Max.   :1.0000   Max.   :0.977  
                                                                       
   p_val_adj            cluster        gene          
 Min.   :0.000e+00   0      : 50   Length:850        
 1st Qu.:0.000e+00   1      : 50   Class :character  
 Median :0.000e+00   2      : 50   Mode  :character  
 Mean   :1.411e-42   3      : 50                     
 3rd Qu.:0.000e+00   4      : 50                     
 Max.   :1.198e-39   5      : 50                     
                     (Other):550              

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
1,0,1.780295,0.383,0.161,0,0,DCT
2,0,1.767947,0.431,0.175,0,0,AMBN
3,0,1.708512,0.464,0.202,0,0,HEY1
4,0,1.623801,0.767,0.311,0,0,LINC01551
5,0,1.543925,0.396,0.189,0,0,OLFM3
6,0,1.514941,0.968,0.510,0,0,SFRP1


## prova con anche negativi

In [124]:
de_genes <- cluster_markers %>% filter(gene %in% genes_of_interest)
de_genes

p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>


In [125]:
cluster_markers_negative <- FindAllMarkers(sc_data_clustered,
                              only.pos = F,   # Considera solo i marker espressi positivamente
                              min.pct = 0.25,    # Percentuale minima di espressione nelle cellule del cluster
                              logfc.threshold = 0.25,  # Soglia minima di LogFC
                              verbose = T)

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16



In [129]:
de_genes_negative <- cluster_markers_negative %>% filter(gene %in% "CIT")
de_genes_negative

p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>


In [128]:
summary(cluster_markers_negative)

     p_val             avg_log2FC            pct.1            pct.2       
 Min.   :0.000e+00   Min.   :-14.37015   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.000e+00   1st Qu.: -0.60813   1st Qu.:0.2700   1st Qu.:0.2510  
 Median :0.000e+00   Median :  0.29920   Median :0.3770   Median :0.3410  
 Mean   :2.483e-04   Mean   :  0.09231   Mean   :0.4394   Mean   :0.4037  
 3rd Qu.:1.000e-09   3rd Qu.:  0.66181   3rd Qu.:0.5810   3rd Qu.:0.5280  
 Max.   :9.999e-03   Max.   : 13.84653   Max.   :1.0000   Max.   :0.9990  
                                                                          
   p_val_adj            cluster          gene          
 Min.   :0.0000000   16     : 1837   Length:25015      
 1st Qu.:0.0000000   8      : 1813   Class :character  
 Median :0.0000000   15     : 1757   Mode  :character  
 Mean   :0.1266954   7      : 1742                     
 3rd Qu.:0.0000316   4      : 1654                     
 Max.   :1.0000000   9      : 1570                     
        

## Save

In [108]:
dir_save = "provv/"

In [121]:
# Seurat Obj
saveRDS(sc_data_clustered, file = paste0(dir_save, file_name, "_SeuratObj.rds"))

In [120]:
# Cluster
write.csv(sc_data_clustered@meta.data, file = paste0(dir_save, file_name, "_SeuratObj_meatadata.csv"))

In [104]:
# PLOTS
for (nm in names(cluster_plots)) {
  ggsave(
    filename = paste0(dir_save,file_name,"_",nm, ".png"),
    plot = cluster_plots[[nm]]
  )
}

Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


In [110]:
# Markers
write.csv(cluster_markers, file = paste0(dir_save,file_name,"_cluster_markers",".csv"))

# Top Genes
write.csv(top_genes, file = paste0(dir_save,file_name,"_top_genes",".csv"))

# Monocle3

# Combine